# Store music metadata using apple shortcuts
In this small project I was trying to store the metadata about the music that gets shazamed on my macOS
I started by trying to find Shazam database file on my system but that proved to be quit tricky
Note that I tried this on MacOs, it might be different in iOS device
TODO: test on mobile device

## Create a Shortcut
the the shortcut app, click on the plus sign, a new shortcut will be created
<img src="imgs/shazam_create.png" alt="create a new shortcut" width="70%" style="margin:5px;"/>


also note that in the video I recorded I used a loop, bad idea since we can run a shortcut in CLI

I modified the short cut to perform one cycle, it takes no input and outputs a song metadata if a song is found as text, otherwise it return some "No song found"

<img src="imgs/shortcut.png" alt="shows the text file created by the shortcut app" width="80%" style="margin:5px;"/>

In [1]:
import os
import re
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SHAZAM_TEMPLATE = """
<?xml version="1.0" encoding="UTF-8"?>
<root>

<timestamp>
Date
</timestamp>

<title>
Shazam Media (Title)
</title>

<artist>
Shazam Media (Artist)
</artist>

<isexplicit>
Shazam Media (Is Explicit)
</isexplicit>

<lyricssnippet>
Shazam Media (Lyrics Snippet)
</lyricssnippet>

<lyricsnippetsynced>
Shazam Media (Lyric Snippet Synced)
</lyricsnippetsynced>

<artwork>
Shazam Media (Artwork)
</artwork>

<videourl>
Shazam Media (Video URL)
</videourl>

<shazamurl>
Shazam Media (Shazam URL)
</shazamurl>

<applemusicurl>
Shazam Media (Apple Music URL)
</applemusicurl>

<name>
Shazam Media (Name)
</name>

</root>
"""

In [3]:
outfile = './test.txt'
res = !shortcuts run "shazam_step" --output-path $outfile

In [4]:
res

[]

find the valid tags then find their values

In [5]:
def parse_row(outfile, encoding='utf-8'):
    """
    The parse_row function parses the XML document using lxml parser.
    it uses the `SHAZAM_TEMPLATE` string as map to read the xml data
    ---------------------------------------------------------
    :param outfile: Specify the path to the xml file that will be parsed
    :return: A dictionary with keys as the tag names
        and values as the text content of those tags
    """
    soup = BeautifulSoup(SHAZAM_TEMPLATE, 'xml')
    root = soup.root
    tags = [tag.name  for tag in root.children if tag and tag.name!='root']
    print()
    try:
        with open(outfile, encoding=encoding) as f:
            row = f.read()
    except FileNotFoundError:
        return FileNotFoundError
    # print(row)
    soup = BeautifulSoup(row.replace('\n', ''), 'xml')
    root = soup.select_one('root')
    if root:
        dct = {}
        for tag in tags:
            if not tag:
                continue
            tag_content = root.find(tag)
            if tag_content:
                dct[tag] = tag_content.text
    return dct

In [6]:
parse_row(outfile)

{'timestamp': '9 May 2024 at 19:56',
 'title': 'Yours',
 'artist': 'JIN',
 'isexplicit': 'No',
 'lyricssnippet': '깊어진 하루 길어진 내 그림자저 멀리 해는 저물고 있어짙어 가는 어둠 속에서 헤메이고 있나 봐이곳에 갇혀버린 걸까아직도 숨 쉬는 곳그곳에 다가가야 해Every day you seem too far away',
 'lyricsnippetsynced': '시간이 멈춰버린 곳이젠 다 되돌려야 해Every day you seem too far awayEvery time you do, I tell myself이곳에서 기다리고 있어Every night I see you in my heart',
 'artwork': 'Image',
 'videourl': '',
 'shazamurl': 'https://www.shazam.com/track/590865488/yours?co=GB=shortcuts=130844=7.092953E-4=264840=2024-05-09T18:56:11.498Z',
 'applemusicurl': 'https://music.apple.com/gb/album/yours/1698507484?i=1698507486=30201=Shazam_shortcuts',
 'name': 'JIN - Yours'}

In [7]:
!cat $outfile

<?xml version="1.0" encoding="UTF-8"?>
<root>

<timestamp>
9 May 2024 at 19:56
</timestamp>

<title>
Yours
</title>

<artist>
JIN
</artist>

<isexplicit>
No
</isexplicit>

<lyricssnippet>
깊어진 하루 길어진 내 그림자
저 멀리 해는 저물고 있어
짙어 가는 어둠 속에서 헤메이고 있나 봐
이곳에 갇혀버린 걸까
아직도 숨 쉬는 곳
그곳에 다가가야 해
Every day you seem too far away
</lyricssnippet>

<lyricsnippetsynced>
시간이 멈춰버린 곳
이젠 다 되돌려야 해
Every day you seem too far away
Every time you do, I tell myself
이곳에서 기다리고 있어
Every night I see you in my heart
</lyricsnippetsynced>

<artwork>
Image
</artwork>

<videourl>

</videourl>

<shazamurl>
https://www.shazam.com/track/590865488/yours?co=GB&referrer=shortcuts&offsetInMilliseconds=130844&timeSkew=7.092953E-4&trackLength=264840&startDate=2024-05-09T18:56:11.498Z
</shazamurl>

<applemusicurl>
https://music.apple.com/gb/album/yours/1698507484?i=1698507486&itscg=30201&itsct=Shazam_shortcuts
</applemusicurl>

<name>
JIN - Yours
</name>

</root>

In [11]:
df = pd.read_json('shazam.json')
isinstance(df,pd.DataFrame)

True